# Superstore EDA — Light Code Notebook

This notebook performs a compact, code-focused Exploratory Data Analysis (EDA) for `Superstore.xlsx`.

Place `Superstore.xlsx` in the same folder as this notebook before running.
Cells contain comments explaining each step; fill interpretation cells manually before submission.

In [ ]:
# --- Imports & Load ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from pathlib import Path

# Load Excel file
file_path = Path("Superstore.xlsx")
if not file_path.exists():
    raise FileNotFoundError("Superstore.xlsx not found in the notebook directory. Please upload it next to this notebook.")

# read all sheets and pick the first sheet
xls = pd.ExcelFile(file_path)
sheet_name = xls.sheet_names[0]
df = pd.read_excel(xls, sheet_name=sheet_name)
print("Loaded sheet:", sheet_name)
print("Shape:", df.shape)
df.head()

In [ ]:
# --- Basic Info & Data Types ---
print("Info:")
print(df.info())
print("\nNumeric columns:")
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(num_cols)
print("\nCategorical columns:")
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(cat_cols)

In [ ]:
# --- Missing Values Summary ---
missing = df.isnull().sum().sort_values(ascending=False)
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.concat([missing, missing_pct], axis=1)
missing_df.columns = ['missing_count', 'missing_pct']
missing_df[missing_df['missing_count']>0]

In [ ]:
# --- Visualize Missing Pattern (simple) ---
plt.figure(figsize=(12,4))
plt.imshow(df.isnull().T, aspect='auto', interpolation='nearest')
plt.xlabel('Row index')
plt.yticks(range(len(df.columns)), df.columns)
plt.title('Missing value matrix (True = missing)')
plt.tight_layout()
plt.show()

In [ ]:
# --- Select Columns for Analysis (auto-suggest) ---
# Auto-suggest: common sales dataset columns; user can customize this list
candidates = {
    'order_id': [c for c in df.columns if 'order' in c.lower() or 'id' in c.lower()],
    'date_cols': [c for c in df.columns if 'date' in c.lower()],
    'customer_cols': [c for c in df.columns if 'customer' in c.lower() or 'segment' in c.lower()],
    'product_cols': [c for c in df.columns if 'product' in c.lower() or 'category' in c.lower()],
    'sales_cols': [c for c in df.columns if any(x in c.lower() for x in ['sales','profit','quantity','discount','amount'])]
}
candidates

In [ ]:
# --- Descriptive Statistics (numeric & categorical) ---
display(df.describe(include='all').T)

In [ ]:
# --- Business Goal & Target Variable ---
# EDIT: Set your business goal and target variable here.
# Example:
# goal = "Predict high-profit orders or identify low-margin product categories"
# target = 'Profit'  # or create a binary target like (df['Profit']>0)
goal = None
target = None

print("Please set 'goal' and 'target' variables in this cell before running modeling steps.")

In [ ]:
# --- Missing Data Handling (examples) ---
# This block shows common imputation strategies. Replace column names as needed.

# Example: numeric imputation with median
numeric_with_na = [c for c in df.select_dtypes(include=[np.number]).columns if df[c].isnull().any()]
for c in numeric_with_na:
    median_val = df[c].median()
    print("Numeric NA:", c, "median ->", median_val)
    df[c + "_preimpute"] = df[c]  # keep copy
    df[c].fillna(median_val, inplace=True)

# Example: categorical imputation with mode
cat_with_na = [c for c in df.select_dtypes(include=['object','category']).columns if df[c].isnull().any()]
for c in cat_with_na:
    mode_val = df[c].mode().iloc[0] if not df[c].mode().empty else "Missing"
    print("Categorical NA:", c, "mode ->", mode_val)
    df[c + "_preimpute"] = df[c]
    df[c].fillna(mode_val, inplace=True)

print('\nPost-imputation missing summary:')
print(df.isnull().sum().sort_values(ascending=False).head())

In [ ]:
# --- Numeric Distributions: histograms and boxplots ---
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# limit to first 6 numeric cols to avoid overload
for c in num_cols[:6]:
    plt.figure(figsize=(10,3))
    plt.subplot(1,2,1)
    plt.hist(df[c].dropna(), bins=30)
    plt.title(f'Histogram: {c}')
    plt.subplot(1,2,2)
    plt.boxplot(df[c].dropna(), vert=False)
    plt.title(f'Boxplot: {c}')
    plt.tight_layout()
    plt.show()

In [ ]:
# --- QQ-plots for normality check (first 4 numeric cols) ---
for c in num_cols[:4]:
    plt.figure(figsize=(6,4))
    stats.probplot(df[c].dropna(), dist='norm', plot=plt)
    plt.title(f'QQ-plot: {c}')
    plt.tight_layout()
    plt.show()

In [ ]:
# --- Outlier Detection (IQR & Z-score) ---
outlier_summary = []
for c in num_cols:
    series = df[c].dropna()
    if series.empty:
        continue
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    iqr_outliers = ((series < lower) | (series > upper)).sum()
    z_scores = np.abs(stats.zscore(series))
    z_outliers = (z_scores > 3).sum()
    outlier_summary.append((c, len(series), iqr_outliers, z_outliers))
outlier_df = pd.DataFrame(outlier_summary, columns=['col','n','iqr_outliers','z_outliers']).sort_values('iqr_outliers', ascending=False)
outlier_df.head(20)

In [ ]:
# --- Correlation Matrix (heatmap) ---
corr = df.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(10,8))
plt.imshow(corr, interpolation='nearest')
plt.colorbar()
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title('Correlation matrix (numeric variables)')
plt.tight_layout()
plt.show()

In [ ]:
# --- Categorical summaries (top categories) ---
for c in cat_cols:
    print('\nColumn:', c)
    print(df[c].value_counts(dropna=False).head(10))
    # simple bar plot for top categories
    top = df[c].value_counts(dropna=False).head(10)
    plt.figure(figsize=(6,3))
    top.plot(kind='bar')
    plt.title(f'Top categories: {c}')
    plt.tight_layout()
    plt.show()

In [ ]:
# --- Scatter plots for numeric relationships (sample) ---
pairs = []
if len(num_cols) >= 2:
    pairs = [(num_cols[i], num_cols[j]) for i in range(min(3,len(num_cols))) for j in range(i+1, min(4,len(num_cols)))]
for x,y in pairs:
    plt.figure(figsize=(5,4))
    plt.scatter(df[x], df[y], alpha=0.6, s=8)
    plt.xlabel(x); plt.ylabel(y)
    plt.title(f'Scatter: {x} vs {y}')
    plt.tight_layout()
    plt.show()

In [ ]:
# --- Save cleaned snapshot (optional) ---
out_path = Path("Superstore_cleaned_snapshot.csv")
df.to_csv(out_path, index=False)
print("Saved cleaned snapshot to", out_path)

## Notes for Submission

- This notebook is **code-only / light**. Add short interpretation text in markdown cells before submitting.
- Customize the 'goal' and 'target' variables and adjust imputation/outlier strategies as needed.
- If the dataset has multiple sheets, the first sheet is used by default.